Explanation and How to Use:

tabu_search(initial_solution, fitness_function, generate_neighborhood, tabu_list_length, max_iterations) Function:

This is the main implementation of the Tabu Search algorithm based on the provided text.

Parameters:

initial_solution: Your starting point (e.g., a list of movie scene indices).

fitness_function: You need to define this function. It calculates the "cost" or "fitness" of a given solution. This is problem-specific. In the text, it's the "total cost of film shooting."

generate_neighborhood: You need to define this function. It creates a list of neighboring solutions from a given solution using 2-opt and 2-swap moves. The example provided is a basic 2-swap, you need to implement the described domain structure.

tabu_list_length: The size of the tabu list. Tune this.

max_iterations: How many iterations to run the search. Tune this.

Logic:

Initializes variables (current solution, best solution, tabu list, etc.).

Iterates for a maximum number of iterations.

Generates candidate solutions (neighbors).

Selects the best candidate that is either not tabu or satisfies the break forbidden level (aspiration).

Updates the current solution, best solution, tabu list, and break forbidden level.

Returns the best solution found and its fitness.

Helper Functions (You MUST Implement/Adapt These):

fitness_function_example(solution):

Replace this with your actual cost calculation for film shooting. This example is just a placeholder that sums the elements of a list.

This function must take a solution (in whatever format you represent it) and return a numerical fitness value (lower value is better for minimization, as assumed in the text).

generate_neighborhood_example(solution):

This is crucial. You need to implement the 2-opt and 2-swap domain structure described in the text. The example is a very basic 2-swap.

2-opt: Involves reversing a section of the solution sequence.

2-swap: Involves swapping two elements in the solution sequence.

Your generate_neighborhood function should create a set of neighbors by applying 2-opt and 2-swap moves to the current solution.

identify_move_example(old_solution, new_solution):

You need to adapt this based on your generate_neighborhood implementation. This function needs to determine the "move" that was made to get from the old_solution to the new_solution.

For 2-swap, it could be the indices that were swapped. For 2-opt, it could be the start and end indices of the reversed segment.

The tabu_list stores these "moves" to prevent reversing them in the near future.

Example Usage (if __name__ == '__main__':)

Provides a simple example of how to use the tabu_search function.

You should replace the initial_solution_example, fitness_function_example, and generate_neighborhood_example with your problem-specific data and functions.

Experiment with tabu_list_length and max_iterations to tune the algorithm for your problem.

To use this for your film scene scheduling problem:

Represent your Film Scene Schedule: Decide how to represent a "solution" (a film scene schedule). A list of scene indices might be a good starting point.

Implement fitness_function: This is the most important step. Write the Python code that calculates the "total cost of film shooting" for a given scene schedule. This will depend on the factors you need to consider in your problem (e.g., location changes, actor availability, set setup times, etc.).

Implement generate_neighborhood: Implement the 2-opt and 2-swap neighborhood generation as described in the text. Make sure it generates valid neighboring schedules.

Implement identify_move: Adapt the identify_move_example to correctly identify the 2-opt or 2-swap move that was made.

Set up Initial Solution: Create a starting initial_solution (e.g., a random scene schedule).

Tune Parameters: Experiment with tabu_list_length and max_iterations to find good values for your problem.

Run and Analyze: Run the tabu_search and analyze the results to see if it's effectively finding better film scene schedules.

In [ ]:
from ortools.linear_solver import pywraplp


def LinearProgrammingExample():
    """Linear programming sample."""
    # Instantiate a Glop solver, naming it LinearExample.
    solver = pywraplp.Solver.CreateSolver("GLOP")
    if not solver:
        return

    # Create the two variables and let them take on any non-negative value.
    x = solver.NumVar(0, solver.infinity(), "x")
    y = solver.NumVar(0, solver.infinity(), "y")

    print("Number of variables =", solver.NumVariables())

    # Constraint 0: x + 2y <= 14.
    solver.Add(x + 2 * y <= 14.0)

    # Constraint 1: 3x - y >= 0.
    solver.Add(3 * x - y >= 0.0)

    # Constraint 2: x - y <= 2.
    solver.Add(x - y <= 2.0)

    print("Number of constraints =", solver.NumConstraints())

    # Objective function: 3x + 4y.
    solver.Maximize(3 * x + 4 * y)

    # Solve the system.
    print(f"Solving with {solver.SolverVersion()}")
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print("Solution:")
        print(f"Objective value = {solver.Objective().Value():0.1f}")
        print(f"x = {x.solution_value():0.1f}")
        print(f"y = {y.solution_value():0.1f}")
    else:
        print("The problem does not have an optimal solution.")

    print("\nAdvanced usage:")
    print(f"Problem solved in {solver.wall_time():d} milliseconds")
    print(f"Problem solved in {solver.iterations():d} iterations")


LinearProgrammingExample()

In [ ]:
from ortools.linear_solver import pywraplp
import numpy as np

# Define solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# Define indices and sets
scenes = range(1, 6)  # Example: 5 scenes
actors = range(1, 4)  # Example: 3 actors
locations = range(1, 3)  # Example: 2 shooting locations

# Parameters (randomly generated for example purposes)
np.random.seed(42)
wages = {p: np.random.randint(80, 101) for p in actors}
scene_durations = {m: np.random.randint(1, 4) for m in scenes}
transfer_costs = {(m, n): np.random.randint(0, 10000) for m in scenes for n in scenes if m != n}

# Decision variables
x = {(m, p): solver.BoolVar(f'x[{m},{p}]') for m in scenes for p in actors}
mu = {(m, n, p): solver.IntVar(0, solver.infinity(), f'mu[{m},{n},{p}]') for m in scenes for n in scenes for p in actors if m != n}
lmbda = {(m, n): solver.BoolVar(f'lambda[{m},{n}]') for m in scenes for n in scenes if m != n}
t = {m: solver.IntVar(0, solver.infinity(), f't[{m}]') for m in scenes}

# Objective function: Minimize total cost
solver.Minimize(solver.Sum(mu[m, n, p] * wages[p] for m in scenes for n in scenes for p in actors if (m, n, p) in mu) +
                solver.Sum(lmbda[m, n] * transfer_costs[m, n] for m in scenes for n in scenes if (m, n) in lmbda))

# Constraints
for m in scenes:
    solver.Add(solver.Sum(lmbda[m, n] for n in scenes if m != n) == 1)
    solver.Add(solver.Sum(lmbda[n, m] for n in scenes if m != n) == 1)

for m in scenes:
    for n in scenes:
        if m != n:
            solver.Add(t[n] - t[m] - scene_durations[m] <= 10000 * (1 - lmbda[m, n]))

for p in actors:
    for m in scenes:
        solver.Add(x[m, p] <= solver.Sum(lmbda[m, n] for n in scenes if m != n))

# Solve the ILP problem
solver.Solve()

# Output results
print("Optimal schedule:")
for m in scenes:
    print(f"Scene {m}: Start time = {t[m].solution_value()}")


In [ ]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum
import numpy as np

# Define indices and sets
scenes = range(1, 6)  # Example: 5 scenes
actors = range(1, 4)  # Example: 3 actors
locations = range(1, 3)  # Example: 2 shooting locations

# Parameters (randomly generated for example purposes)
np.random.seed(42)
wages = {p: np.random.randint(80, 101) for p in actors}
scene_durations = {m: np.random.randint(1, 4) for m in scenes}
transfer_costs = {(m, n): np.random.randint(0, 10000) for m in scenes for n in scenes if m != n}

# Decision variables
prob = LpProblem("MovieSceneScheduling", LpMinimize)
x = LpVariable.dicts("x", [(m, p) for m in scenes for p in actors], cat="Binary")
mu = LpVariable.dicts("mu", [(m, n, p) for m in scenes for n in scenes for p in actors if m != n], lowBound=0, cat="Integer")
lmbda = LpVariable.dicts("lambda", [(m, n) for m in scenes for n in scenes if m != n], cat="Binary")
t = LpVariable.dicts("t", scenes, lowBound=0, cat="Integer")

# Objective function: Minimize total cost
prob += lpSum(mu[m, n, p] * wages[p] for m in scenes for n in scenes for p in actors if (m, n, p) in mu) + \
        lpSum(lmbda[m, n] * transfer_costs[m, n] for m in scenes for n in scenes if (m, n) in lmbda)

# Constraints
for m in scenes:
    prob += lpSum(lmbda[m, n] for n in scenes if m != n) == 1
    prob += lpSum(lmbda[n, m] for n in scenes if m != n) == 1

for m in scenes:
    for n in scenes:
        if m != n:
            prob += t[n] - t[m] - scene_durations[m] <= 10000 * (1 - lmbda[m, n])

for p in actors:
    for m in scenes:
        prob += x[m, p] <= lpSum(lmbda[m, n] for n in scenes if m != n)

# Solve the ILP problem
prob.solve()

# Output results
print("Optimal schedule:")
for m in scenes:
    print(f"Scene {m}: Start time = {t[m].varValue}")


In [ ]:
import random
from collections import deque

def tabu_search(initial_solution, fitness_function, generate_neighborhood, tabu_list_length, max_iterations):
    """
    Implements the Tabu Search algorithm as described in the text.

    Args:
        initial_solution: The starting solution (e.g., a list representing movie scene sequence).
        fitness_function: A function that takes a solution and returns its fitness value (total cost).
        generate_neighborhood: A function that takes a solution and returns a list of neighbor solutions.
                               This should implement the 2-opt and 2-swap domain movement.
        tabu_list_length: The maximum length of the tabu list.
        max_iterations: The maximum number of iterations to run the algorithm.

    Returns:
        best_solution: The best solution found during the search.
        best_fitness: The fitness value of the best solution.
    """

    current_solution = initial_solution
    best_solution = initial_solution
    best_fitness = fitness_function(best_solution)
    tabu_list = deque(maxlen=tabu_list_length) # Use deque for efficient FIFO
    iteration_count = 0
    break_forbidden_level = best_fitness # Initialize break forbidden level with initial best fitness

    while iteration_count < max_iterations:
        iteration_count += 1
        candidate_solutions = generate_neighborhood(current_solution)
        best_candidate = None
        best_candidate_fitness = float('inf') # Initialize with positive infinity for minimization

        for candidate in candidate_solutions:
            fitness = fitness_function(candidate)
            move = identify_move(current_solution, candidate) # Need to define identify_move based on your neighborhood structure

            is_tabu = False
            if move in tabu_list:
                is_tabu = True

            # Step 3: Check Break Forbidden Level
            if fitness < break_forbidden_level:
                best_candidate = candidate
                best_candidate_fitness = fitness
                break_forbidden_level = fitness # Update break forbidden level
                is_tabu = False # Override tabu status due to aspiration

            # Step 4: Select non-tabu best solution if break forbidden level not met
            elif not is_tabu and fitness < best_candidate_fitness:
                best_candidate = candidate
                best_candidate_fitness = fitness


        # Step 4 & 5: Choose the best candidate as current solution (if found)
        if best_candidate is not None:
            current_solution = best_candidate

            # Step 6: Update Global Best Solution
            current_fitness = fitness_function(current_solution)
            if current_fitness < best_fitness:
                best_fitness = current_fitness
                best_solution = current_solution

            # Step 6: Update Tabu List
            move_to_tabu = identify_move(initial_solution, current_solution) # Assuming move is relative to the initial solution for simplicity. Adjust if needed.
            if move_to_tabu is not None: # Ensure move is identified (in case neighborhood generation doesn't always produce a valid move)
                tabu_list.append(move_to_tabu)


        else:
            # Step 5: Handle case where no improving non-tabu solution is found (optional - can add diversification here if needed)
            # For now, we simply continue with the current_solution, effectively exploring from the same point again.
            pass # Or implement diversification strategy here as described in the text if needed


    return best_solution, best_fitness


# --- Helper Functions (Adapt these to your specific problem) ---

def fitness_function_example(solution):
    """
    Example fitness function - replace with your actual cost calculation for film shooting.
    This example just sums the elements of the solution list. Lower sum is better.
    """
    return sum(solution)

def generate_neighborhood_example(solution):
    """
    Example neighborhood generation using 2-swap.
    Generates a small neighborhood by performing a few random 2-swaps.
    You should implement 2-opt and 2-swap as described in the text.
    """
    neighborhood = []
    n = len(solution)
    for _ in range(5): # Generate a neighborhood of 5 solutions (adjust as needed)
        neighbor = solution[:] # Create a copy to avoid modifying original
        if n > 1:
            idx1, idx2 = random.sample(range(n), 2)
            neighbor[idx1], neighbor[idx2] = neighbor[idx2], neighbor[idx1]
        neighborhood.append(neighbor)
    return neighborhood

def identify_move_example(old_solution, new_solution):
    """
    Example identify_move function for 2-swap.
    Identifies the indices that were swapped to get from old_solution to new_solution.
    This is a simplified example and might need adjustment based on your neighborhood generation.
    For 2-swap, we can assume only two positions changed.
    """
    diff_indices = []
    for i in range(len(old_solution)):
        if old_solution[i] != new_solution[i]:
            diff_indices.append(i)
    if len(diff_indices) == 2:
        return tuple(sorted(diff_indices)) # Return the swapped indices as a tuple (tabu move)
    return None # If move cannot be identified or is not a simple 2-swap


# --- Example Usage ---
if __name__ == '__main__':
    # Example problem: Minimize the sum of elements in a list

    initial_solution_example = [10, 5, 8, 2, 9, 1, 7, 4, 6, 3]

    best_sol, best_fit = tabu_search(
        initial_solution=initial_solution_example,
        fitness_function=fitness_function_example,
        generate_neighborhood=generate_neighborhood_example,
        tabu_list_length=7, # Example tabu list length - tune this parameter
        max_iterations=100 # Example max iterations - tune this parameter
    )

    print("Best Solution Found:", best_sol)
    print("Best Fitness Value:", best_fit)

Explanation and How to Use:

particle_swarm_optimization(num_particles, max_iterations, initial_solution_generator, fitness_function) Function:

This is the core PSO implementation.

Parameters:

num_particles: Number of particles in the swarm (Num).

max_iterations: Maximum iterations (G).

initial_solution_generator: You need to define this function. It should generate a random starting solution (scene sequence). The example initial_solution_generator_example_pso creates a random permutation of scenes.

fitness_function: You need to define this function. It calculates the fitness (total cost) of a solution. The example fitness_function_example_pso is a placeholder.

Logic:

Initializes particles, personal bests (pbest), and global best (gbest).

Iterates for max_iterations.

In each iteration:

Evaluates fitness of each particle.

Updates pbest if a particle finds a better position.

Updates gbest if any particle finds a globally better position.

Updates particle positions using the update_particle_position function (explained below).

Returns the gbest solution and fitness.

update_particle_position(current_position, personal_best_position, global_best_position, ...) Function:

This is where you need to adapt the PSO update rule for your sequence problem.

Simplified Implementation: The provided code uses a simplified approach for updating particle positions in a sequence (permutation) context. It does not use traditional velocity vectors directly added to positions (as in continuous PSO). Instead, it uses swaps to move a particle's position towards its pbest and gbest positions.

Key Idea: It iterates through the scene indices and, with certain probabilities influenced by cognitive_coefficient and social_coefficient, tries to swap scenes in the current_position to match the order in personal_best_position and global_best_position. It also adds a small probability of random swaps for exploration.

Important: You might need to refine this update_particle_position function based on the specifics of your film scheduling problem and how you want to move from one scene sequence to another in a PSO manner. Consider more sophisticated sequence manipulation techniques if needed.

Helper Functions (You MUST Implement/Adapt These):

fitness_function_example_pso(solution):

Replace this with your actual cost calculation for film shooting. This is the same as for Tabu Search – it needs to calculate the total cost for a given scene schedule.

initial_solution_generator_example_pso(num_scenes=10):

Adapt this to generate a valid initial random scene sequence for your problem. The example creates a random permutation of scenes from 1 to num_scenes.

Example Usage (if __name__ == '__main__':)

Shows how to use the particle_swarm_optimization function.

You need to replace fitness_function_example_pso and initial_solution_generator_example_pso with your problem-specific implementations.

Tune num_particles, max_iterations, and the PSO parameters (inertia_weight, cognitive_coefficient, social_coefficient) to get good performance.

To use this for your film scene scheduling problem:

Represent Film Scene Schedule: Same as for Tabu Search, decide how to represent a solution (scene schedule). A list of scene indices is a good starting point.

Implement fitness_function: Write the Python code to calculate the "total cost of film shooting" for a given scene schedule. This function is crucial and problem-specific.

Implement initial_solution_generator: Create a function that generates a valid random initial scene schedule.

Refine update_particle_position: Carefully consider how to update particle positions (scene sequences) in a way that effectively explores the search space and moves particles towards better solutions (guided by pbest and gbest). The provided simplified swap-based method might be a starting point, but you might need to develop a more sophisticated update mechanism.

Tune Parameters: Experiment with num_particles, max_iterations, inertia_weight, cognitive_coefficient, and social_coefficient to find good parameter settings for your problem.

Run and Analyze: Run the particle_swarm_optimization and analyze the results. Compare its performance with Tabu Search and other methods if applicable.

In [ ]:
import random

def particle_swarm_optimization(num_particles, max_iterations, initial_solution_generator, fitness_function):
    """
    Implements the Particle Swarm Optimization algorithm as described in the text.

    Args:
        num_particles: The number of particles in the swarm (Num).
        max_iterations: The maximum number of iterations (G).
        initial_solution_generator: A function that generates a random initial solution (scene sequence).
        fitness_function: A function that calculates the fitness (total cost) of a solution.

    Returns:
        global_best_solution: The best solution found by the PSO algorithm.
        global_best_fitness: The fitness value of the best solution.
    """

    particles = []
    particle_velocities = [] # In this simplified implementation, velocity is more conceptual
    personal_best_positions = []
    personal_best_fitnesses = []

    # Step 2: Initialize the particle swarm
    for _ in range(num_particles):
        initial_position = initial_solution_generator()
        particles.append(initial_position)
        particle_velocities.append(None) # Velocity concept is simplified for sequence problems
        personal_best_positions.append(initial_position)
        personal_best_fitnesses.append(fitness_function(initial_position))

    global_best_solution = personal_best_positions[0]
    global_best_fitness = personal_best_fitnesses[0]

    # Initialize PSO parameters (you can tune these)
    inertia_weight = 0.7
    cognitive_coefficient = 1.4
    social_coefficient = 1.4

    # Step 1: Set iteration counter
    iteration_count = 0

    while iteration_count < max_iterations:
        iteration_count += 1

        for i in range(num_particles):
            current_position = particles[i]

            # Step 3: Calculate fitness value
            current_fitness = fitness_function(current_position)

            # Step 4: Local comparison (pbest update)
            if current_fitness < personal_best_fitnesses[i]:
                personal_best_fitnesses[i] = current_fitness
                personal_best_positions[i] = current_position[:] # Copy to avoid modification

            # Step 5: Global comparison (gbest update)
            if current_fitness < global_best_fitness:
                global_best_fitness = current_fitness
                global_best_solution = current_position[:] # Copy to avoid modification

        # Step 6: Update position and "velocity" (simplified for sequence)
        for i in range(num_particles):
            new_position = update_particle_position(
                current_position=particles[i],
                personal_best_position=personal_best_positions[i],
                global_best_position=global_best_solution,
                inertia_weight=inertia_weight,
                cognitive_coefficient=cognitive_coefficient,
                social_coefficient=social_coefficient
            )
            particles[i] = new_position

        # Step 7: Check termination condition is implicitly handled by the loop condition

    return global_best_solution, global_best_fitness


def update_particle_position(current_position, personal_best_position, global_best_position,
                             inertia_weight, cognitive_coefficient, social_coefficient):
    """
    Updates a particle's position based on PSO principles for a sequence problem.
    This is a simplified implementation. You might need to adjust the update mechanism
    based on your specific film scheduling problem and solution representation.

    Here, we use a combination of inertia (keeping some of the current order) and
    influence from personal best and global best to generate a new position.

    A simple approach is to randomly swap scenes, with probabilities influenced by
    how the current position differs from pbest and gbest.
    """
    new_position = current_position[:] # Start with the current position

    # Introduce inertia: keep some of the current order (simplified by default start)

    # Influence from personal best and global best:
    # We can make swaps based on differences between current, pbest and gbest

    indices_to_swap = list(range(len(current_position)))
    random.shuffle(indices_to_swap) # Randomize the order of indices to consider for swaps


    for idx in indices_to_swap:
        if random.random() < cognitive_coefficient / (cognitive_coefficient + social_coefficient): # Favor pbest influence
            if current_position[idx] != personal_best_position[idx]: # If different from pbest at this index
                # Try to move towards pbest
                pbest_val_at_idx = personal_best_position[idx]
                current_val_at_idx = current_position[idx]

                try:
                    pbest_idx_current_val = current_position.index(pbest_val_at_idx) # Find where pbest's scene is in current
                    new_position[idx], new_position[pbest_idx_current_val] = new_position[pbest_idx_current_val], new_position[idx] # Swap to move closer to pbest
                except ValueError:
                    pass # Scene not found (shouldn't happen in a valid permutation but handle for robustness)


        else: # Favor gbest influence
            if current_position[idx] != global_best_position[idx]: # If different from gbest at this index
                # Try to move towards gbest
                gbest_val_at_idx = global_best_position[idx]
                current_val_at_idx = current_position[idx]

                try:
                    gbest_idx_current_val = current_position.index(gbest_val_at_idx) # Find where gbest's scene is in current
                    new_position[idx], new_position[gbest_idx_current_val] = new_position[gbest_idx_current_val], new_position[idx] # Swap to move closer to gbest
                except ValueError:
                    pass # Scene not found (shouldn't happen in a valid permutation but handle for robustness)

        # Add some randomness (exploration) - can adjust probability
        if random.random() < 0.1: # Small chance to make a random swap for exploration
            swap_idx = random.choice(indices_to_swap)
            if swap_idx != idx:
                new_position[idx], new_position[swap_idx] = new_position[swap_idx], new_position[idx]


    return new_position


# --- Helper Functions (Adapt these to your specific problem) ---

def fitness_function_example_pso(solution):
    """
    Example fitness function for PSO - replace with your actual cost calculation.
    """
    return sum(solution)

def initial_solution_generator_example_pso(num_scenes=10):
    """
    Example initial solution generator - creates a random scene sequence.
    """
    scenes = list(range(1, num_scenes + 1)) # Scenes numbered 1 to num_scenes
    random.shuffle(scenes)
    return scenes


# --- Example Usage ---
if __name__ == '__main__':
    num_particles = 30
    max_iterations = 100
    num_scenes_example = 10 # Adjust number of scenes for your problem

    best_solution_pso, best_fitness_pso = particle_swarm_optimization(
        num_particles=num_particles,
        max_iterations=max_iterations,
        initial_solution_generator=lambda: initial_solution_generator_example_pso(num_scenes_example),
        fitness_function=fitness_function_example_pso
    )

    print("PSO Best Solution Found:", best_solution_pso)
    print("PSO Best Fitness Value:", best_fitness_pso)

Explanation and How to Use:

ant_colony_optimization(...) Function:

Main ACO function, orchestrates the algorithm.

Parameters:

num_ants: Number of ants (M).

max_iterations: Maximum iterations (Num).

num_scenes: Number of scenes to schedule.

fitness_function: You MUST implement this. Calculates the total cost of a scene schedule.

heuristic_function: You MUST implement this. Provides heuristic information to guide ants' choices.

evaporation_rate, alpha, beta: ACO parameters (tune these).

Logic:

Initializes pheromones, best solution variables.

Iterates for max_iterations.

In each iteration:

Each ant constructs a path (scene schedule) using construct_ant_path.

Calculates the fitness (cost) of each path.

Updates pheromones using update_pheromones, based on the best path found in the current iteration.

Returns the best_solution and best_fitness.

initialize_pheromones(...) Function:

Sets up the initial pheromone levels between all pairs of scenes. Initially set to a small constant value.

construct_ant_path(...) Function:

A single ant constructs a complete scene schedule.

Starts at a start_scene.

Iteratively chooses the next_scene using choose_next_scene until all scenes are in the path.

choose_next_scene(...) Function:

Probabilistically selects the next scene for an ant.

Calculates probabilities based on:

pheromone_levels: Pheromone trail between the current_scene and possible next_scenes.

heuristic_function: Heuristic value for the transition from current_scene to next_scenes.

alpha and beta: Control the influence of pheromone and heuristic.

Uses roulette wheel selection (or similar) to choose the next_scene based on the calculated probabilities.

update_pheromones(...) Function:

Updates pheromone levels after each iteration.

Pheromone Evaporation: Decreases pheromone levels on all paths by evaporation_rate.

Pheromone Deposition: Increases pheromone levels on the path of the best ant in the current iteration. The amount of pheromone deposited is inversely proportional to the cost of the best path.

Helper Functions (You MUST Implement/Adapt These):

fitness_function_example_aco(solution):

Replace with your actual cost calculation for film shooting. Same as for Tabu Search and PSO.

heuristic_function_example_aco(current_scene, next_scene):

This is crucial for ACO. You must design a good heuristic function that guides ants towards promising scene transitions. The example is very basic (prefers smaller scene index differences).

A good heuristic should incorporate problem-specific knowledge (e.g., minimize location changes, actor availability conflicts, etc.).

Example Usage (if __name__ == '__main__':)

Shows how to use the ant_colony_optimization function.

Replace fitness_function_example_aco and heuristic_function_example_aco with your problem-specific implementations.

Tune ACO parameters (num_ants, max_iterations, evaporation_rate, alpha, beta) to get good performance.

To use this for your film scene scheduling problem:

Represent Film Scene Schedule: Use a list of scene indices to represent a schedule.

Implement fitness_function: Write the code to calculate the "total cost of film shooting" for a given scene schedule.

Implement heuristic_function: This is key for ACO's success. Design a heuristic function that estimates the "desirability" of transitioning from one scene to another. This function should reflect factors that reduce cost in your problem (e.g., location similarity, actor availability).

Tune Parameters: Experiment with num_ants, max_iterations, evaporation_rate, alpha, and beta to find good parameter values for your problem.

Run and Analyze: Run ant_colony_optimization and analyze the results. Compare its performance to Tabu Search and PSO if you've implemented those as well.

In [ ]:
import random

def ant_colony_optimization(num_ants, max_iterations, num_scenes, fitness_function, heuristic_function,
                             evaporation_rate=0.5, alpha=1, beta=2):
    """
    Implements the Ant Colony Optimization algorithm as described in the text.

    Args:
        num_ants: The total number of ants (M).
        max_iterations: The maximum number of iterations (Num).
        num_scenes: The number of scenes to schedule.
        fitness_function: A function that calculates the fitness (total cost) of a scene schedule.
        heuristic_function: A function that provides heuristic information for scene transitions.
                             It should take the current scene and a potential next scene as input.
        evaporation_rate: Pheromone evaporation coefficient (rho).
        alpha: Pheromone factor.
        beta: Heuristic factor.

    Returns:
        best_solution: The best scene schedule found by the ACO algorithm.
        best_fitness: The fitness value of the best solution.
    """

    # Step 1: Initialization
    pheromone_levels = initialize_pheromones(num_scenes) # Pheromone levels between scene transitions
    best_solution = None
    best_fitness = float('inf')
    iteration_count = 0

    while iteration_count < max_iterations:
        iteration_count += 1
        ant_paths = [] # Store paths (scene schedules) for each ant
        ant_path_costs = []

        # Step 2: Initialize each ant and plan path
        for ant_index in range(num_ants):
            start_scene = random.randint(0, num_scenes - 1) # Random starting scene (0-indexed)
            path = construct_ant_path(start_scene, num_scenes, pheromone_levels, heuristic_function, alpha, beta)
            ant_paths.append(path)

        # Step 3: Calculate objective function and update pheromones
        for path in ant_paths:
            cost = fitness_function(path) # Calculate cost for each ant's path
            ant_path_costs.append(cost)
            if cost < best_fitness:
                best_fitness = cost
                best_solution = path[:] # Copy the path

        pheromone_levels = update_pheromones(pheromone_levels, ant_paths, ant_path_costs, evaporation_rate, best_solution, fitness_function)

        # Step 4: Check termination condition (iteration limit) - handled by loop condition

    return best_solution, best_fitness


def initialize_pheromones(num_scenes, initial_pheromone=1.0):
    """
    Initializes pheromone levels for all possible scene transitions.

    Args:
        num_scenes: The number of scenes.
        initial_pheromone: The initial pheromone level for each transition.

    Returns:
        A 2D list (or dictionary) representing pheromone levels between scenes.
        pheromone_levels[i][j] is the pheromone level from scene i to scene j.
    """
    pheromone_levels = [[initial_pheromone] * num_scenes for _ in range(num_scenes)]
    return pheromone_levels


def construct_ant_path(start_scene, num_scenes, pheromone_levels, heuristic_function, alpha, beta):
    """
    Constructs a path (scene schedule) for a single ant.

    Args:
        start_scene: The starting scene for the ant.
        num_scenes: The total number of scenes.
        pheromone_levels: The current pheromone levels.
        heuristic_function: The heuristic function.
        alpha: Pheromone factor.
        beta: Heuristic factor.

    Returns:
        A list representing the scene schedule (path) constructed by the ant.
    """
    path = []
    visited_scenes = set()
    current_scene = start_scene
    path.append(current_scene)
    visited_scenes.add(current_scene)

    while len(visited_scenes) < num_scenes:
        next_scene = choose_next_scene(current_scene, num_scenes, visited_scenes, pheromone_levels, heuristic_function, alpha, beta)
        if next_scene is None: # Handle case where no valid next scene is found (shouldn't happen in typical ACO if pheromones are initialized correctly and heuristics are reasonable)
            break # Or implement a recovery strategy
        path.append(next_scene)
        visited_scenes.add(next_scene)
        current_scene = next_scene
    return path


def choose_next_scene(current_scene, num_scenes, visited_scenes, pheromone_levels, heuristic_function, alpha, beta):
    """
    Probabilistically chooses the next scene for an ant based on pheromone and heuristic.

    Args:
        current_scene: The current scene the ant is at.
        num_scenes: The total number of scenes.
        visited_scenes: Set of scenes already visited in the current path.
        pheromone_levels: The current pheromone levels.
        heuristic_function: The heuristic function.
        alpha: Pheromone factor.
        beta: Heuristic factor.

    Returns:
        The index of the next scene to visit, or None if no valid next scene.
    """
    probabilities = []
    possible_scenes = [scene for scene in range(num_scenes) if scene not in visited_scenes]

    if not possible_scenes:
        return None # No more scenes to visit

    denominator = 0
    for next_scene in possible_scenes:
        pheromone = pheromone_levels[current_scene][next_scene]
        heuristic_value = heuristic_function(current_scene, next_scene)
        denominator += (pheromone**alpha) * (heuristic_value**beta)

    if denominator == 0: # Handle division by zero (shouldn't happen if heuristics are reasonable)
        return random.choice(possible_scenes) # Fallback to random choice

    for next_scene in possible_scenes:
        pheromone = pheromone_levels[current_scene][next_scene]
        heuristic_value = heuristic_function(current_scene, next_scene)
        numerator = (pheromone**alpha) * (heuristic_value**beta)
        probabilities.append(numerator / denominator)

    # Roulette wheel selection (or similar probabilistic selection)
    cumulative_probability = 0.0
    rand_val = random.random()
    for i in range(len(possible_scenes)):
        cumulative_probability += probabilities[i]
        if rand_val <= cumulative_probability:
            return possible_scenes[i]
    return possible_scenes[-1] # Should not reach here in normal cases


def update_pheromones(pheromone_levels, ant_paths, ant_path_costs, evaporation_rate, best_solution, fitness_function):
    """
    Updates pheromone levels based on ant paths and their costs.

    Args:
        pheromone_levels: The current pheromone levels.
        ant_paths: List of paths (scene schedules) constructed by ants in the current iteration.
        ant_path_costs: List of costs corresponding to each ant path.
        evaporation_rate: Pheromone evaporation coefficient.
        best_solution: The best solution found so far in the current iteration (or globally).
        fitness_function: Fitness function to recalculate best solution cost if needed.

    Returns:
        Updated pheromone levels.
    """
    # Pheromone evaporation
    for i in range(len(pheromone_levels)):
        for j in range(len(pheromone_levels[i])):
            pheromone_levels[i][j] *= (1.0 - evaporation_rate) # Evaporation

    # Pheromone deposition - based on best solution in current iteration (or you can use all ants, or best ant)
    best_path_index = ant_path_costs.index(min(ant_path_costs)) # Index of ant with best path in current iteration
    best_path_current_iteration = ant_paths[best_path_index]
    pheromone_deposit_amount = 1.0 / fitness_function(best_path_current_iteration) # Pheromone deposit inversely proportional to cost

    for i in range(len(best_path_current_iteration) - 1):
        current_scene = best_path_current_iteration[i]
        next_scene = best_path_current_iteration[i+1]
        pheromone_levels[current_scene][next_scene] += pheromone_deposit_amount # Deposit pheromone on path of best ant


    return pheromone_levels


# --- Helper Functions (Adapt these to your specific problem) ---

def fitness_function_example_aco(solution):
    """
    Example fitness function for ACO - replace with your actual cost calculation.
    """
    return sum(solution) # Example: sum of scene indices (lower is better in this example)

def heuristic_function_example_aco(current_scene, next_scene):
    """
    Example heuristic function for ACO - replace with your problem-specific heuristic.
    This example prioritizes moving to scenes with lower index values (just for demonstration).
    A good heuristic should guide ants towards promising transitions based on problem knowledge.
    """
    return 1.0 / (abs(next_scene - current_scene) + 1)  # Example: Prefer smaller scene index differences


# --- Example Usage ---
if __name__ == '__main__':
    num_ants = 50
    max_iterations = 100
    num_scenes_example = 10

    best_solution_aco, best_fitness_aco = ant_colony_optimization(
        num_ants=num_ants,
        max_iterations=max_iterations,
        num_scenes=num_scenes_example,
        fitness_function=fitness_function_example_aco,
        heuristic_function=heuristic_function_example_aco
    )

    print("ACO Best Solution Found:", best_solution_aco)
    print("ACO Best Fitness Value:", best_fitness_aco)

In [ ]:
import time
import random
from tabulate import tabulate # For nice table formatting (install: pip install tabulate)

# --- Import your Algorithm Implementations from previous responses ---
from collections import deque

def tabu_search(initial_solution, fitness_function, generate_neighborhood, tabu_list_length, max_iterations):
    # ... (Your TSBM implementation from previous response) ...
    current_solution = initial_solution
    best_solution = initial_solution
    best_fitness = fitness_function(best_solution)
    tabu_list = deque(maxlen=tabu_list_length)
    iteration_count = 0
    break_forbidden_level = best_fitness

    while iteration_count < max_iterations:
        iteration_count += 1
        candidate_solutions = generate_neighborhood(current_solution)
        best_candidate = None
        best_candidate_fitness = float('inf')

        for candidate in candidate_solutions:
            fitness = fitness_function(candidate)
            move = identify_move_mssp(current_solution, candidate) # Use MSSP specific identify_move

            is_tabu = False
            if move in tabu_list:
                is_tabu = True

            if fitness < break_forbidden_level:
                best_candidate = candidate
                best_candidate_fitness = fitness
                break_forbidden_level = fitness
                is_tabu = False
            elif not is_tabu and fitness < best_candidate_fitness:
                best_candidate = candidate
                best_candidate_fitness = fitness

        if best_candidate is not None:
            current_solution = best_candidate
            current_fitness = fitness_function(current_solution)
            if current_fitness < best_fitness:
                best_fitness = current_fitness
                best_solution = current_solution

            move_to_tabu = identify_move_mssp(initial_solution, current_solution) # Use MSSP specific identify_move
            if move_to_tabu is not None:
                tabu_list.append(move_to_tabu)
        else:
            pass

    return best_solution, best_fitness

def particle_swarm_optimization(num_particles, max_iterations, initial_solution_generator, fitness_function):
    # ... (Your PSOBM implementation from previous response) ...
    particles = []
    particle_velocities = []
    personal_best_positions = []
    personal_best_fitnesses = []

    for _ in range(num_particles):
        initial_position = initial_solution_generator()
        particles.append(initial_position)
        particle_velocities.append(None)
        personal_best_positions.append(initial_position)
        personal_best_fitnesses.append(fitness_function(initial_position))

    global_best_solution = personal_best_positions[0]
    global_best_fitness = personal_best_fitnesses[0]

    inertia_weight = 0.7
    cognitive_coefficient = 1.4
    social_coefficient = 1.4

    iteration_count = 0

    while iteration_count < max_iterations:
        iteration_count += 1

        for i in range(num_particles):
            current_position = particles[i]
            current_fitness = fitness_function(current_position)

            if current_fitness < personal_best_fitnesses[i]:
                personal_best_fitnesses[i] = current_fitness
                personal_best_positions[i] = current_position[:]

            if current_fitness < global_best_fitness:
                global_best_fitness = current_fitness
                global_best_solution = current_position[:]

        for i in range(num_particles):
            new_position = update_particle_position_mssp( # Use MSSP specific update_particle_position
                current_position=particles[i],
                personal_best_position=personal_best_positions[i],
                global_best_position=global_best_solution,
                inertia_weight=inertia_weight,
                cognitive_coefficient=cognitive_coefficient,
                social_coefficient=social_coefficient
            )
            particles[i] = new_position

    return global_best_solution, global_best_fitness

def ant_colony_optimization(num_ants, max_iterations, num_scenes, fitness_function, heuristic_function,
                             evaporation_rate=0.5, alpha=1, beta=2):
    # ... (Your ACOBM implementation from previous response) ...
    pheromone_levels = initialize_pheromones_mssp(num_scenes) # Use MSSP specific initialize_pheromones
    best_solution = None
    best_fitness = float('inf')
    iteration_count = 0

    while iteration_count < max_iterations:
        iteration_count += 1
        ant_paths = []
        ant_path_costs = []

        for ant_index in range(num_ants):
            start_scene = random.randint(0, num_scenes - 1)
            path = construct_ant_path_mssp(start_scene, num_scenes, pheromone_levels, heuristic_function, alpha, beta) # Use MSSP specific construct_ant_path
            ant_paths.append(path)

        for path in ant_paths:
            cost = fitness_function(path)
            ant_path_costs.append(cost)
            if cost < best_fitness:
                best_fitness = cost
                best_solution = path[:]

        pheromone_levels = update_pheromones_mssp(pheromone_levels, ant_paths, ant_path_costs, evaporation_rate, best_solution, fitness_function) # Use MSSP specific update_pheromones

    return best_solution, best_fitness

# --- MSSP Problem Specific Functions ---

def generate_mssp_instance(num_scenes, num_actors, num_locations):
    """Generates a random instance of the Movie Scene Scheduling Problem."""
    scene_transfer_costs = [[0] * num_scenes for _ in range(num_scenes)]
    for i in range(num_scenes):
        for j in range(i + 1, num_scenes):
            cost = random.randint(0, 10000)
            scene_transfer_costs[i][j] = scene_transfer_costs[j][i] = cost

    actor_daily_wages = [random.randint(80, 100) for _ in range(num_actors)]
    scene_locations = [random.randint(0, num_locations - 1) for _ in range(num_scenes)] # 0-indexed locations
    actor_scene_presence = [[random.choice([0, 1]) for _ in range(num_scenes)] for _ in range(num_actors)] # Binary: actor p in scene m?
    shooting_times = [random.randint(1, 5) for _ in range(num_scenes)] # Example shooting times (days)
    transition_times = [[random.randint(1, 3) for _ in range(num_scenes)] for _ in range(num_scenes)] # Example transition times (days)
    for i in range(num_scenes):
        for j in range(num_scenes):
            if scene_locations[i] == scene_locations[j]:
                transition_times[i][j] = 0

    return {
        "num_scenes": num_scenes,
        "num_actors": num_actors,
        "num_locations": num_locations,
        "scene_transfer_costs": scene_transfer_costs,
        "actor_daily_wages": actor_daily_wages,
        "scene_locations": scene_locations,
        "actor_scene_presence": actor_scene_presence,
        "shooting_times": shooting_times,
        "transition_times": transition_times,
    }

def msp_fitness_function(solution, instance_data):
    """Calculates the total cost of a movie scene schedule."""
    transfer_cost = 0
    for i in range(len(solution) - 1):
        transfer_cost += instance_data["scene_transfer_costs"][solution[i]][solution[i+1]]

    actor_wage_cost = 0
    actor_working_days = [0] * instance_data["num_actors"]

    current_day = 0
    location_schedule = {} # Track scenes at each location and their start days

    for scene_index in solution:
        location = instance_data["scene_locations"][scene_index]
        shooting_time = instance_data["shooting_times"][scene_index]

        if location not in location_schedule:
            location_schedule[location] = []

        location_schedule[location].append({'scene': scene_index, 'start_day': current_day, 'end_day': current_day + shooting_time})
        current_day += shooting_time # Scenes at the same location shot consecutively

    # Calculate actor working days based on location schedules
    for actor_id in range(instance_data["num_actors"]):
        max_end_day = 0
        for location in location_schedule:
            location_scenes = location_schedule[location]
            actor_scenes_at_location = [scene_data for scene_data in location_scenes if instance_data["actor_scene_presence"][actor_id][scene_data['scene']]]

            if actor_scenes_at_location: # Actor is in scenes at this location
                first_scene_day = min(scene_data['start_day'] for scene_data in actor_scenes_at_location)
                last_scene_day = max(scene_data['end_day'] for scene_data in actor_scenes_at_location)
                max_end_day = max(max_end_day, last_scene_day) # Track latest day actor works

        actor_working_days[actor_id] = max_end_day

    for actor_id in range(instance_data["num_actors"]):
        actor_wage_cost += actor_working_days[actor_id] * instance_data["actor_daily_wages"][actor_id]

    return transfer_cost + actor_wage_cost

def generate_initial_solution_mssp(num_scenes):
    """Generates a random initial scene schedule."""
    scenes = list(range(num_scenes))
    random.shuffle(scenes)
    return scenes

def generate_neighborhood_mssp(solution):
    """Generates neighbors using 2-opt and 2-swap for MSSP."""
    neighbors = []
    # 2-swap
    for i in range(len(solution)):
        for j in range(i + 1, len(solution)):
            neighbor_swap = solution[:]
            neighbor_swap[i], neighbor_swap[j] = neighbor_swap[j], neighbor_swap[i]
            neighbors.append(neighbor_swap)
    # 2-opt
    for i in range(len(solution)):
        for j in range(i + 1, len(solution)):
            neighbor_opt = solution[:]
            neighbor_opt[i:j+1] = reversed(neighbor_opt[i:j+1])
            neighbors.append(neighbor_opt)
    return neighbors

def identify_move_mssp(old_solution, new_solution):
    """Identifies the move (2-swap or 2-opt) made.""" # Simplified for demonstration
    if len(old_solution) != len(new_solution):
        return None
    diff_indices = []
    for i in range(len(old_solution)):
        if old_solution[i] != new_solution[i]:
            diff_indices.append(i)

    if len(diff_indices) == 2:
        return tuple(sorted(diff_indices)) # 2-swap move
    elif len(diff_indices) > 2: # Heuristic for 2-opt detection (can be improved)
        if sorted(old_solution) == sorted(new_solution): # Check if it's a reversal (permutation check)
            return "2-opt-like-move" # Not precise 2-opt identification but enough for tabu in this example
    return None

def update_particle_position_mssp(current_position, personal_best_position, global_best_position,
                             inertia_weight, cognitive_coefficient, social_coefficient):
    """Simplified PSO position update for MSSP (similar to previous PSO)."""
    new_position = current_position[:]

    indices_to_swap = list(range(len(current_position)))
    random.shuffle(indices_to_swap)

    for idx in indices_to_swap:
        if random.random() < cognitive_coefficient / (cognitive_coefficient + social_coefficient):
            if current_position[idx] != personal_best_position[idx]:
                pbest_val_at_idx = personal_best_position[idx]
                try:
                    pbest_idx_current_val = current_position.index(pbest_val_at_idx)
                    new_position[idx], new_position[pbest_idx_current_val] = new_position[pbest_idx_current_val], new_position[idx]
                except ValueError:
                    pass
        else:
            if current_position[idx] != global_best_position[idx]:
                gbest_val_at_idx = global_best_position[idx]
                try:
                    gbest_idx_current_val = current_position.index(gbest_val_at_idx)
                    new_position[idx], new_position[gbest_idx_current_val] = new_position[gbest_idx_current_val], new_position[idx]
                except ValueError:
                    pass

        if random.random() < 0.1:
            swap_idx = random.choice(indices_to_swap)
            if swap_idx != idx:
                new_position[idx], new_position[swap_idx] = new_position[swap_idx], new_position[idx]

    return new_position

def initialize_pheromones_mssp(num_scenes, initial_pheromone=1.0):
    """Initializes pheromones for MSSP."""
    return [[initial_pheromone] * num_scenes for _ in range(num_scenes)]

def construct_ant_path_mssp(start_scene, num_scenes, pheromone_levels, heuristic_function, alpha, beta):
    """Constructs ant path for MSSP."""
    path = []
    visited_scenes = set()
    current_scene = start_scene
    path.append(current_scene)
    visited_scenes.add(current_scene)

    while len(visited_scenes) < num_scenes:
        next_scene = choose_next_scene_mssp(current_scene, num_scenes, visited_scenes, pheromone_levels, heuristic_function, alpha, beta)
        if next_scene is None:
            break
        path.append(next_scene)
        visited_scenes.add(next_scene)
        current_scene = next_scene
    return path

def choose_next_scene_mssp(current_scene, num_scenes, visited_scenes, pheromone_levels, heuristic_function, alpha, beta):
    """Chooses next scene for ACO in MSSP."""
    probabilities = []
    possible_scenes = [scene for scene in range(num_scenes) if scene not in visited_scenes]

    if not possible_scenes:
        return None

    denominator = 0
    for next_scene in possible_scenes:
        pheromone = pheromone_levels[current_scene][next_scene]
        heuristic_value = heuristic_function(current_scene, next_scene)
        denominator += (pheromone**alpha) * (heuristic_value**beta)

    if denominator == 0:
        return random.choice(possible_scenes)

    for next_scene in possible_scenes:
        pheromone = pheromone_levels[current_scene][next_scene]
        heuristic_value = heuristic_function(current_scene, next_scene)
        numerator = (pheromone**alpha) * (heuristic_value**beta)
        probabilities.append(numerator / denominator)

    cumulative_probability = 0.0
    rand_val = random.random()
    for i in range(len(possible_scenes)):
        cumulative_probability += probabilities[i]
        if rand_val <= cumulative_probability:
            return possible_scenes[i]
    return possible_scenes[-1]

def update_pheromones_mssp(pheromone_levels, ant_paths, ant_path_costs, evaporation_rate, best_solution, fitness_function):
    """Updates pheromones for MSSP."""
    for i in range(len(pheromone_levels)):
        for j in range(len(pheromone_levels[i])):
            pheromone_levels[i][j] *= (1.0 - evaporation_rate)

    best_path_index = ant_path_costs.index(min(ant_path_costs))
    best_path_current_iteration = ant_paths[best_path_index]
    pheromone_deposit_amount = 1.0 / fitness_function(best_path_current_iteration)

    for i in range(len(best_path_current_iteration) - 1):
        current_scene = best_path_current_iteration[i]
        next_scene = best_path_current_iteration[i+1]
        pheromone_levels[current_scene][next_scene] += pheromone_deposit_amount

    return pheromone_levels

def msp_heuristic_function(current_scene, next_scene, instance_data):
    """Heuristic for MSSP: Prefer scenes at the same location or lower transfer cost."""
    if instance_data["scene_locations"][current_scene] == instance_data["scene_locations"][next_scene]:
        return 2.0  # Higher heuristic for same location
    else:
        return 1.0 / (instance_data["scene_transfer_costs"][current_scene][next_scene] / 1000 + 1) # Lower cost is better

# --- Experiment Pipeline ---

def run_experiment(algorithm_name, algorithm_func, instance_data, algorithm_params):
    """Runs a single experiment for a given algorithm and instance."""
    start_time = time.time()
    best_solution, best_fitness = algorithm_func(**algorithm_params)
    end_time = time.time()
    runtime = end_time - start_time
    return best_fitness, runtime

def experiment_pipeline(cases, num_instances_per_case=3): # Increased instances for better averaging
    """Executes the experiment pipeline for different cases and algorithms."""
    results_data = []

    for case_id, (num_scenes, num_actors, num_locations) in enumerate(cases):
        print(f"--- Case ID: {case_id+1} (Scenes: {num_scenes}, Actors: {num_actors}, Locations: {num_locations}) ---")
        for instance_index in range(num_instances_per_case):
            print(f"  --- Instance: {instance_index+1} ---")
            instance_data = generate_mssp_instance(num_scenes, num_actors, num_locations)

            # --- TSBM ---
            tsbm_params = {
                "initial_solution": generate_initial_solution_mssp(num_scenes),
                "fitness_function": lambda sol: msp_fitness_function(sol, instance_data),
                "generate_neighborhood": generate_neighborhood_mssp,
                "tabu_list_length": 10, # Tunable parameter
                "max_iterations": 100 # Tunable parameter
            }
            objt, tt = run_experiment("TSBM", tabu_search, instance_data, tsbm_params)

            # --- PSOBM ---
            psobm_params = {
                "num_particles": 30, # Tunable parameter
                "max_iterations": 100, # Tunable parameter
                "initial_solution_generator": lambda: generate_initial_solution_mssp(num_scenes),
                "fitness_function": lambda sol: msp_fitness_function(sol, instance_data),
            }
            objp, tp = run_experiment("PSOBM", particle_swarm_optimization, instance_data, psobm_params)

            # --- ACOBM ---
            acobm_params = {
                "num_ants": 50, # Tunable parameter
                "max_iterations": 100, # Tunable parameter
                "num_scenes": num_scenes,
                "fitness_function": lambda sol: msp_fitness_function(sol, instance_data),
                "heuristic_function": lambda current, next_scene: msp_heuristic_function(current, next_scene, instance_data),
                "evaporation_rate": 0.5, # Tunable parameter
                "alpha": 1, # Tunable parameter
                "beta": 2, # Tunable parameter
            }
            obja, ta = run_experiment("ACOBM", ant_colony_optimization, instance_data, acobm_params)

            results_data.append({
                "Case ID": f"{num_scenes}-{num_actors}-{num_locations}-{instance_index+1}",
                "OBJT": objt,
                "OBJP": objp,
                "OBJA": obja,
                "TT(s)": tt,
                "TP(s)": tp,
                "TA(s)": ta,
                "Gap1": (objt - obja) / obja * 100 if obja != 0 else float('inf'), # Gap % vs ACOBM
                "Gap2": (objp - obja) / obja * 100 if obja != 0 else float('inf'), # Gap % vs ACOBM
            })

    # --- Analyze and Print Results ---
    average_results = {}
    case_ids = set(data["Case ID"].split('-')[:-1][0] + '-' + data["Case ID"].split('-')[:-1][1] + '-' + data["Case ID"].split('-')[:-1][2] for data in results_data) # Group by base case ID

    for base_case_id in case_ids:
        case_data_points = [data for data in results_data if base_case_id in data["Case ID"]]
        avg_objt = sum(data["OBJT"] for data in case_data_points) / len(case_data_points)
        avg_objp = sum(data["OBJP"] for data in case_data_points) / len(case_data_points)
        avg_obja = sum(data["OBJA"] for data in case_data_points) / len(case_data_points)
        avg_tt = sum(data["TT(s)"] for data in case_data_points) / len(case_data_points)
        avg_tp = sum(data["TP(s)"] for data in case_data_points) / len(case_data_points)
        avg_ta = sum(data["TA(s)"] for data in case_data_points) / len(case_data_points)
        avg_gap1 = sum(data["Gap1"] for data in case_data_points) / len(case_data_points)
        avg_gap2 = sum(data["Gap2"] for data in case_data_points) / len(case_data_points)

        average_results[base_case_id] = {
            "OBJT": avg_objt,
            "OBJP": avg_objp,
            "OBJA": avg_obja,
            "TT(s)": avg_tt,
            "TP(s)": avg_tp,
            "TA(s)": avg_ta,
            "Gap1": avg_gap1,
            "Gap2": avg_gap2,
        }

    table_headers = ["Case ID", "OBJT", "OBJP", "OBJA", "TT(s)", "TP(s)", "TA(s)", "Gap1 (%)", "Gap2 (%)"]
    table_data = []
    for case_id_base in sorted(average_results.keys(), key=lambda x: list(map(int, x.split('-')))): # Sort numerically
        avg_res = average_results[case_id_base]
        table_data.append([
            case_id_base,
            f"{avg_res['OBJT']:.2f}",
            f"{avg_res['OBJP']:.2f}",
            f"{avg_res['OBJA']:.2f}",
            f"{avg_res['TT(s)']:.2f}",
            f"{avg_res['TP(s)']:.2f}",
            f"{avg_res['TA(s)']:.2f}",
            f"{avg_res['Gap1']:.2f}",
            f"{avg_res['Gap2']:.2f}",
        ])

    print("\n--- Average Results Table ---")
    print(tabulate(table_data, headers=table_headers, tablefmt="grid"))

    # --- Comparison of TSBM and PSOBM (TABLE 2 style) ---
    table2_data = []
    for case_id_base in sorted(average_results.keys(), key=lambda x: list(map(int, x.split('-')))):
        avg_res = average_results[case_id_base]
        gap3 = (avg_res['OBJP'] - avg_res['OBJT']) / avg_res['OBJT'] * 100 if avg_res['OBJT'] != 0 else float('inf')
        table2_data.append([
            case_id_base,
            f"{avg_res['OBJT']:.2f}",
            f"{avg_res['OBJP']:.2f}",
            f"{gap3:.2f}"
        ])

    table2_headers = ["Case ID", "OBJT", "OBJP", "Gap3 (%)"]
    print("\n--- Comparison of TSBM and PSOBM (TABLE 2 Style) ---")
    print(tabulate(table2_data, headers=table2_headers, tablefmt="grid"))


# --- Main Execution ---
if __name__ == "__main__":
    # Define experiment cases (Scene-Actor-Location scales)
    experiment_cases = [
        (10, 5, 2),   # Case 1: Small scale
        (20, 10, 3),  # Case 2: Medium scale
        (30, 15, 4),  # Case 3: Larger scale
        (20, 10, 3),  # Case 4: Repeat of Case 2 to test variance (different instance)
        (10, 5, 2),   # Case 5: Repeat of Case 1
        (30, 15, 4),  # Case 6: Repeat of Case 3
    ]

    experiment_pipeline(experiment_cases, num_instances_per_case=5) # Run experiment with defined cases and instances

New changes

In [ ]:
import time
import random
from tabulate import tabulate
import pandas as pd
import sys
from collections import deque
from ortools.linear_solver import pywraplp

# --- Algorithm Implementations (TSBM, PSOBM, ACOBM) ---
def tabu_search(initial_solution, fitness_function, generate_neighborhood, tabu_list_length, max_iterations):
    # ... (Your TSBM implementation from previous response) ...
    current_solution = initial_solution
    best_solution = initial_solution
    best_fitness = fitness_function(best_solution)
    tabu_list = deque(maxlen=tabu_list_length)
    iteration_count = 0
    break_forbidden_level = best_fitness

    while iteration_count < max_iterations:
        iteration_count += 1
        candidate_solutions = generate_neighborhood(current_solution)
        best_candidate = None
        best_candidate_fitness = float('inf')

        for candidate in candidate_solutions:
            fitness = fitness_function(candidate)
            move = identify_move_mssp(current_solution, candidate) # Use MSSP specific identify_move

            is_tabu = False
            if move in tabu_list:
                is_tabu = True

            if fitness < break_forbidden_level:
                best_candidate = candidate
                best_candidate_fitness = fitness
                break_forbidden_level = fitness
                is_tabu = False
            elif not is_tabu and fitness < best_candidate_fitness:
                best_candidate = candidate
                best_candidate_fitness = fitness

        if best_candidate is not None:
            current_solution = best_candidate
            current_fitness = fitness_function(current_solution)
            if current_fitness < best_fitness:
                best_fitness = current_fitness
                best_solution = current_solution

            move_to_tabu = identify_move_mssp(initial_solution, current_solution) # Use MSSP specific identify_move
            if move_to_tabu is not None:
                tabu_list.append(move_to_tabu)
        else:
            pass

    return best_solution, best_fitness

def particle_swarm_optimization(num_particles, max_iterations, initial_solution_generator, fitness_function):
    # ... (Your PSOBM implementation from previous response) ...
    particles = []
    particle_velocities = []
    personal_best_positions = []
    personal_best_fitnesses = []

    for _ in range(num_particles):
        initial_position = initial_solution_generator()
        particles.append(initial_position)
        particle_velocities.append(None)
        personal_best_positions.append(initial_position)
        personal_best_fitnesses.append(fitness_function(initial_position))

    global_best_solution = personal_best_positions[0]
    global_best_fitness = personal_best_fitnesses[0]

    inertia_weight = 0.7
    cognitive_coefficient = 1.4
    social_coefficient = 1.4

    iteration_count = 0

    while iteration_count < max_iterations:
        iteration_count += 1

        for i in range(num_particles):
            current_position = particles[i]
            current_fitness = fitness_function(current_position)

            if current_fitness < personal_best_fitnesses[i]:
                personal_best_fitnesses[i] = current_fitness
                personal_best_positions[i] = current_position[:]

            if current_fitness < global_best_fitness:
                global_best_fitness = current_fitness
                global_best_solution = current_position[:]

        for i in range(num_particles):
            new_position = update_particle_position_mssp( # Use MSSP specific update_particle_position
                current_position=particles[i],
                personal_best_position=personal_best_positions[i],
                global_best_position=global_best_solution,
                inertia_weight=inertia_weight,
                cognitive_coefficient=cognitive_coefficient,
                social_coefficient=social_coefficient
            )
            particles[i] = new_position

    return global_best_solution, global_best_fitness

def ant_colony_optimization(num_ants, max_iterations, num_scenes, fitness_function, heuristic_function,
                             evaporation_rate=0.5, alpha=1, beta=2):
    # ... (Your ACOBM implementation from previous response) ...
    pheromone_levels = initialize_pheromones_mssp(num_scenes) # Use MSSP specific initialize_pheromones
    best_solution = None
    best_fitness = float('inf')
    iteration_count = 0

    while iteration_count < max_iterations:
        iteration_count += 1
        ant_paths = []
        ant_path_costs = []

        for ant_index in range(num_ants):
            start_scene = random.randint(0, num_scenes - 1)
            path = construct_ant_path_mssp(start_scene, num_scenes, pheromone_levels, heuristic_function, alpha, beta) # Use MSSP specific construct_ant_path
            ant_paths.append(path)

        for path in ant_paths:
            cost = fitness_function(path)
            ant_path_costs.append(cost)
            if cost < best_fitness:
                best_fitness = cost
                best_solution = path[:]

        pheromone_levels = update_pheromones_mssp(pheromone_levels, ant_paths, ant_path_costs, evaporation_rate, best_solution, fitness_function) # Use MSSP specific update_pheromones

    return best_solution, best_fitness

# --- MSSP Problem Specific Functions ---
def generate_mssp_instance(instance_id,
                      # Rango por defecto para instancias aleatorias
                      min_scenes=8, max_scenes=40,
                      min_actors=8, max_actors=20,
                      min_locations=5, max_locations=20,
                      min_days_factor=1.0, max_days_factor=3.0):
    """
    Genera una instancia del problema de manera aleatoria.
    """
    n = random.randint(min_scenes, max_scenes)
    m = random.randint(min_actors, max_actors)
    num_locations = random.randint(min_locations, max_locations)

    # Duraciones de las escenas
    scene_durations = {f"s{j}": random.randint(1, 5) for j in range(n)}
    total_duration = sum(scene_durations.values())
    p = random.randint(int(min_days_factor * total_duration),
                       int(max_days_factor * total_duration))

    # Salarios de actores
    actor_wages = {f"a{i}": random.randint(100, 1000) for i in range(m)}

    # Participación de actores (actor_participation[a][s] = 0/1)
    actor_participation = {}
    for i in range(m):
        participation = {f"s{j}": random.choices([0, 1], weights=[0.7, 0.3])[0]
                         for j in range(n)}
        actor_participation[f"a{i}"] = participation

    # Asegurar que cada escena tenga al menos un actor
    for j in range(n):
        while sum(actor_participation[f"a{i}"][f"s{j}"] for i in range(m)) == 0:
            actor_participation[f"a{random.randint(0, m-1)}"][f"s{j}"] = 1

    # Costos de locación
    location_costs = {
        f"l{k}": {
            day: random.randint(500, 5000) for day in range(p)
        } for k in range(num_locations)
    }

    instance_data = {
        "instance_id": instance_id,
        "num_scenes": n,
        "scene_durations": scene_durations,
        "actor_wages": actor_wages,
        "actor_participation": actor_participation,
        "location_costs": location_costs,
        "planning_horizon": p
    }

    return instance_data

class ScheduleSolution:
    def __init__(self, sequence, start_days):
        self.sequence = sequence  # Lista de índices de escenas (enteros)
        self.start_days = start_days  # Dict { "s0": día_inicio, "s1": ... }

def compute_actor_cost(solution, instance):
    actor_days = {actor: set() for actor in instance["actor_wages"]}
    for scene_idx in solution.sequence:
        scene_id = f"s{scene_idx}"
        start = solution.start_days[scene_id]
        duration = instance["scene_durations"][scene_id]
        for actor, parts in instance["actor_participation"].items():
            if parts[scene_id]:
                for day in range(start, start + duration):
                    actor_days[actor].add(day)
    total = 0
    for actor, days in actor_days.items():
        if days:
            cost = (max(days) - min(days) + 1) * instance["actor_wages"][actor]
            total += cost
    return total

def compute_location_cost(solution, instance):
    total = 0
    for loc in instance["location_costs"]:
        for scene_idx in solution.sequence:
            scene_id = f"s{scene_idx}"
            start = solution.start_days[scene_id]
            duration = instance["scene_durations"][scene_id]
            for day in range(start, start + duration):
                total += instance["location_costs"][loc][day]
    return total

def total_cost(solution, instance):
    return compute_actor_cost(solution, instance) + compute_location_cost(solution, instance)

def solve_subproblem(sequence, instance):
    """
    Dado un orden fijo de escenas, determina la asignación óptima de días
    para minimizar costos de locación (usando OR-Tools/SCIP).
    """
    solver = pywraplp.Solver.CreateSolver('SCIP')
    horizon = instance["planning_horizon"]

    scenes = [f"s{i}" for i in sequence]
    start_vars = {
        scene: solver.IntVar(0, horizon - instance["scene_durations"][scene], f"start_{scene}")
        for scene in scenes
    }

    # Respetar el orden
    for i in range(len(scenes) - 1):
        curr = scenes[i]
        nxt = scenes[i + 1]
        solver.Add(start_vars[nxt] >= start_vars[curr] + instance["scene_durations"][curr])

    # Función objetivo: costos de locación
    objective = solver.Objective()
    for scene in scenes:
        duration = instance["scene_durations"][scene]
        for loc in instance["location_costs"]:
            for d in range(horizon):
                if d + duration <= horizon:
                    cost = sum(instance["location_costs"][loc][d + k] for k in range(duration))
                    objective.SetCoefficient(start_vars[scene], cost)
    objective.SetMinimization()

    status = solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        start_days = {scene: int(start_vars[scene].solution_value()) for scene in scenes}
        return ScheduleSolution(sequence, start_days)
    else:
        return None

def relocate_move(solution, instance):
    seq = solution.sequence.copy()
    if len(seq) < 2: return solution
    i = random.randint(0, len(seq)-1)
    scene = seq.pop(i)
    j = random.randint(0, len(seq))
    seq.insert(j, scene)
    new_sol = solve_subproblem(seq, instance)
    return new_sol if new_sol else solution

def swap_move(solution, instance):
    seq = solution.sequence.copy()
    if len(seq) < 2: return solution
    i, j = random.sample(range(len(seq)), 2)
    seq[i], seq[j] = seq[j], seq[i]
    new_sol = solve_subproblem(seq, instance)
    return new_sol if new_sol else solution

def local_search(current_solution, instance, max_no_improve=10):
    best = current_solution
    best_cost = total_cost(best, instance)
    no_improve = 0
    while no_improve < max_no_improve:
        neighbor = random.choice([relocate_move, swap_move])(best, instance)
        c_neighbor = total_cost(neighbor, instance)
        if c_neighbor < best_cost:
            best = neighbor
            best_cost = c_neighbor
            no_improve = 0
        else:
            no_improve += 1
    return best

def perturb(solution, instance):
    seq = solution.sequence.copy()
    if random.random() < 0.5:
        # 2-opt
        i = random.randint(0, len(seq)-2)
        j = random.randint(i+1, len(seq)-1)
        seq[i:j+1] = reversed(seq[i:j+1])
    else:
        # Swaps aleatorios
        for _ in range(random.randint(1, 4)):
            i, j = random.sample(range(len(seq)), 2)
            seq[i], seq[j] = seq[j], seq[i]
    new_sol = solve_subproblem(seq, instance)
    return new_sol if new_sol else solution

def initial_solution(instance):
    scenes = list(range(instance["num_scenes"]))
    random.shuffle(scenes)
    return solve_subproblem(scenes, instance)

def iterated_local_search(instance, max_iter, delta_ini, epsilon):
    current = initial_solution(instance)
    best = current
    delta = delta_ini
    for _ in range(max_iter):
        perturbed = perturb(current, instance)
        candidate = local_search(perturbed, instance)
        cost_curr = total_cost(current, instance)
        cost_cand = total_cost(candidate, instance)
        if cost_cand < cost_curr * (1 + delta):
            current = candidate
            if cost_cand < total_cost(best, instance):
                best = candidate
        delta *= epsilon
    return best

def solve_milp_sim(instance):
    """
    Simula la parte exacta usando la secuencia natural (0,1,2,...) como si fuera una solución MILP.
    Retorna costo, gap=0 y tiempo de ejecución.
    """
    start_time = time.time()
    seq = list(range(instance["num_scenes"]))
    solution = solve_subproblem(seq, instance)
    if solution:
        cost = total_cost(solution, instance)
    else:
        cost = float('inf')
    elapsed = time.time() - start_time
    return cost, 0.0, elapsed


def run_experiment(instance_id, ils_params=None, runs_ils=1): # Reduced runs_ils for faster execution in example
    if ils_params is None:
        ils_params = {'max_iter': 100, 'delta_ini': 0.1, 'epsilon': 0.9} # Example ILS parameters

    # Generar instancia aleatoria
    inst = generate_mssp_instance(instance_id)
    num_scenes = inst["num_scenes"]

    # 1) "MILP" simulado
    milp_start = time.time()
    milp_Z, milp_gap, _ = solve_milp_sim(inst)
    milp_time = time.time() - milp_start

    # --- 2) TSBM ---
    tsbm_start = time.time()
    tsbm_best_solution, objt = tabu_search(
        initial_solution=generate_initial_solution_mssp(num_scenes),
        fitness_function=lambda sol: msp_fitness_function_ortools(sol, inst), # Use ortools compatible fitness
        generate_neighborhood=generate_neighborhood_mssp,
        tabu_list_length=10,
        max_iterations=100
    )
    tsbm_time = time.time() - tsbm_start

    # --- 3) PSOBM ---
    psobm_start = time.time()
    psobm_best_solution, objp = particle_swarm_optimization(
        num_particles=30,
        max_iterations=100,
        initial_solution_generator=lambda: generate_initial_solution_mssp(num_scenes),
        fitness_function=lambda sol: msp_fitness_function_ortools(sol, inst), # Use ortools compatible fitness
    )
    psobm_time = time.time() - psobm_start

    # --- 4) ACOBM ---
    acobm_start = time.time()
    acobm_best_solution, obja = ant_colony_optimization(
        num_ants=50,
        max_iterations=100,
        num_scenes=num_scenes,
        fitness_function=lambda sol: msp_fitness_function_ortools(sol, inst), # Use ortools compatible fitness
        heuristic_function=lambda current, next_scene: msp_heuristic_function_ortools(current, next_scene, inst) # Use ortools compatible heuristic
    )
    acobm_time = time.time() - acobm_start


    # 5) ILS (Iterated Local Search) - using provided ILS as baseline
    ils_costs = []
    ils_times = []
    for _ in range(runs_ils):
        start_ils = time.time()
        sol = iterated_local_search(inst,
                                    max_iter=ils_params["max_iter"],
                                    delta_ini=ils_params["delta_ini"],
                                    epsilon=ils_params["epsilon"])
        cost_ils = total_cost(sol, inst)
        ils_costs.append(cost_ils)
        ils_times.append(time.time() - start_ils)

    Zbest_ils = min(ils_costs)
    avg_ils_time = sum(ils_times) / len(ils_times)

    # gap'(%) = 100*((Zbest_ils - milp_Z)/milp_Z)
    if milp_Z != 0:
        ils_gap_prime = 100.0 * (Zbest_ils - milp_Z) / milp_Z
    else:
        ils_gap_prime = None

    # gap promedio (%) de las 10 corridas
    # gap_i = 100*((c / Zbest_ils) - 1)
    gap_list = [100.0 * (c / Zbest_ils - 1) for c in ils_costs]
    avg_gap = sum(gap_list) / len(gap_list)


    return {
        "id": instance_id,
        "n": num_scenes,
        "m": len(inst["actor_wages"]),
        "p": inst["planning_horizon"],
        "Z(ILP)": round(milp_Z, 2), # Renamed to ILP to match script
        "gap(ILP)(%)": round(milp_gap, 2), # Renamed to ILP to match script
        "time(ILP)(s)": round(milp_time, 2), # Renamed to ILP to match script
        "Z(ILS)": round(Zbest_ils, 2), # Added ILS results
        "gap'(ILS)(%)": round(ils_gap_prime, 2) if ils_gap_prime is not None else None, # Added ILS gap
        "gap(ILS)(%)": round(avg_gap, 2), # Added ILS avg gap
        "time(ILS)(s)": round(avg_ils_time, 2), # Added ILS time
        "OBJT": round(objt, 2),
        "OBJP": round(objp, 2),
        "OBJA": round(obja, 2),
        "TT(s)": round(tsbm_time, 2),
        "TP(s)": round(psobm_time, 2),
        "TA(s)": round(acobm_time, 2),
        "Gap1": (objt - obja) / obja * 100 if obja != 0 else float('inf'), # Gap % vs ACOBM
        "Gap2": (objp - obja) / obja * 100 if obja != 0 else float('inf'), # Gap % vs ACOBM
        "Gap3": (objp - objt) / objt * 100 if objt != 0 else float('inf'), # Gap % TSBM vs PSOBM
        "Gap4": (objt - milp_Z) / milp_Z * 100 if milp_Z != 0 else float('inf'), # Gap % TSBM vs ILP
        "Gap5": (objp - milp_Z) / milp_Z * 100 if milp_Z != 0 else float('inf'), # Gap % PSOBM vs ILP
        "Gap6": (obja - milp_Z) / milp_Z * 100 if milp_Z != 0 else float('inf'), # Gap % ACOBM vs ILP
    }

def experiment_pipeline(cases, num_instances_per_case=1): # Reduced instances for faster example
    """Executes the experiment pipeline for different cases and algorithms."""
    results_data = []

    for case_id, (min_scenes, max_scenes, min_actors, max_actors, min_locations, max_locations) in enumerate(cases):
        print(f"--- Case ID: {case_id+1} (Scenes Range: {min_scenes}-{max_scenes}, Actors Range: {min_actors}-{max_actors}, Locations Range: {min_locations}-{max_locations}) ---")
        for instance_index in range(num_instances_per_case):
            print(f"  --- Instance: {instance_index+1} ---")
            instance_data = generate_mssp_instance(
                instance_id=f"{case_id+1}-{instance_index+1}",
                min_scenes=min_scenes, max_scenes=max_scenes,
                min_actors=min_actors, max_actors=max_actors,
                min_locations=min_locations, max_locations=max_locations
            )

            instance_results = run_experiment(instance_id=f"{case_id+1}-{instance_index+1}", runs_ils=1) # Reduced runs_ils for faster example
            results_data.append(instance_results)


    # --- Analyze and Print Results - TABLE 1 Style (Comparison to ACOBM and ILP) ---
    average_results_table1 = {}
    case_ids_table1 = set(data["id"].split('-')[0] for data in results_data) # Group by base case ID

    for base_case_id in case_ids_table1:
        case_data_points = [data for data in results_data if data["id"].startswith(base_case_id + '-')]
        avg_objt = sum(data["OBJT"] for data in case_data_points) / len(case_data_points)
        avg_objp = sum(data["OBJP"] for data in case_data_points) / len(case_data_points)
        avg_obja = sum(data["OBJA"] for data in case_data_points) / len(case_data_points)
        avg_tt = sum(data["TT(s)"] for data in case_data_points) / len(case_data_points)
        avg_tp = sum(data["TP(s)"] for data in case_data_points) / len(case_data_points)
        avg_ta = sum(data["TA(s)"] for data in case_data_points) / len(case_data_points)
        avg_ilp = sum(data["Z(ILP)"] for data in case_data_points) / len(case_data_points)
        avg_gap1 = sum(data["Gap1"] for data in case_data_points) / len(case_data_points) # vs ACOBM
        avg_gap2 = sum(data["Gap2"] for data in case_data_points) / len(case_data_points) # vs ACOBM
        avg_gap4 = sum(data["Gap4"] for data in case_data_points) / len(case_data_points) # vs ILP
        avg_gap5 = sum(data["Gap5"] for data in case_data_points) / len(case_data_points) # vs ILP
        avg_gap6 = sum(data["Gap6"] for data in case_data_points) / len(case_data_points) # vs ILP


        average_results_table1[base_case_id] = {
            "OBJT": avg_objt,
            "OBJP": avg_objp,
            "OBJA": avg_obja,
            "Z(ILP)": avg_ilp,
            "TT(s)": avg_tt,
            "TP(s)": avg_tp,
            "TA(s)": avg_ta,
            "Gap1": avg_gap1, # vs ACOBM
            "Gap2": avg_gap2, # vs ACOBM
            "Gap4": avg_gap4, # vs ILP
            "Gap5": avg_gap5, # vs ILP
            "Gap6": avg_gap6, # vs ILP
        }

    table1_headers = ["Case ID", "Z(ILP)", "OBJT", "OBJP", "OBJA", "TT(s)", "TP(s)", "TA(s)", "Gap1 (%)", "Gap2 (%)", "Gap4 (%)", "Gap5 (%)", "Gap6 (%)"] # Added ILP and gaps
    table1_data = []
    for case_id_base in sorted(average_results_table1.keys(), key=int):
        avg_res = average_results_table1[case_id_base]
        table1_data.append([
            case_id_base,
            f"{avg_res['Z(ILP)']:.2f}",
            f"{avg_res['OBJT']:.2f}",
            f"{avg_res['OBJP']:.2f}",
            f"{avg_res['OBJA']:.2f}",
            f"{avg_res['TT(s)']:.2f}",
            f"{avg_res['TP(s)']:.2f}",
            f"{avg_res['TA(s)']:.2f}",
            f"{avg_res['Gap1']:.2f}", # vs ACOBM
            f"{avg_res['Gap2']:.2f}", # vs ACOBM
            f"{avg_res['Gap4']:.2f}", # vs ILP
            f"{avg_res['Gap5']:.2f}", # vs ILP
            f"{avg_res['Gap6']:.2f}", # vs ILP
        ])

    print("\n--- Average Results Table 1 (Comparison to ACOBM and ILP) ---")
    print(tabulate(table1_data, headers=table1_headers, tablefmt="grid"))


    # --- Analyze and Print Results - TABLE 2 Style (Comparison of TSBM, PSOBM and ILS) ---
    average_results_table2 = {}
    case_ids_table2 = set(data["id"].split('-')[0] for data in results_data) # Group by base case ID

    for base_case_id in case_ids_table2:
        case_data_points = [data for data in results_data if data["id"].startswith(base_case_id + '-')]
        avg_objt = sum(data["OBJT"] for data in case_data_points) / len(case_data_points)
        avg_objp = sum(data["OBJP"] for data in case_data_points) / len(case_data_points)
        avg_ils = sum(data["Z(ILS)"] for data in case_data_points) / len(case_data_points)
        average_results_table2[base_case_id] = {
            "OBJT": avg_objt,
            "OBJP": avg_objp,
            "Z(ILS)": avg_ils,
        }

    table2_headers = ["Case ID", "Z(ILS)", "OBJT", "OBJP", "Gap3 (%)"] # Added ILS
    table2_data = []
    for case_id_base in sorted(average_results_table2.keys(), key=int):
        avg_res = average_results_table2[case_id_base]
        gap3 = (avg_res['OBJP'] - avg_res['OBJT']) / avg_res['OBJT'] * 100 if avg_res['OBJT'] != 0 else float('inf')
        table2_data.append([
            case_id_base,
            f"{avg_res['Z(ILS)']:.2f}", # ILS
            f"{avg_res['OBJT']:.2f}",
            f"{avg_res['OBJP']:.2f}",
            f"{gap3:.2f}"
        ])

    print("\n--- Average Results Table 2 (Comparison of TSBM, PSOBM and ILS) ---")
    print(tabulate(table2_data, headers=table2_headers, tablefmt="grid"))


# --- Fitness and Heuristic Functions adapted for ortools instance format ---
def msp_fitness_function_ortools(solution, instance_data):
    """Fitness function adapted for ortools instance format."""
    # Convert solution from scene indices to scene IDs (s0, s1, ...)
    solution_scene_ids = [f"s{scene_index}" for scene_index in solution]
    schedule_solution = ScheduleSolution(solution_scene_ids, {}) # Dummy start_days, will be recalculated
    # Need to calculate start_days for the given sequence to evaluate cost correctly
    optimized_schedule = solve_subproblem(solution, instance_data) # Solve subproblem to get optimal start days
    if optimized_schedule:
        return total_cost(optimized_schedule, instance_data)
    else:
        return float('inf') # Or handle no feasible schedule case

def msp_heuristic_function_ortools(current_scene_index, next_scene_index, instance_data):
    """Heuristic function adapted for ortools instance - example heuristic, adapt as needed."""
    current_scene_id = f"s{current_scene_index}"
    next_scene_id = f"s{next_scene_index}"
    # Example: Simple heuristic based on scene durations (shorter scenes preferred earlier - adapt as needed)
    return 1.0 / (instance_data["scene_durations"][next_scene_id] + 0.1) # Adding small value to avoid division by zero


def generate_initial_solution_mssp(num_scenes):
    """Generates a random initial scene schedule for MSSP (using scene indices)."""
    scenes = list(range(num_scenes))
    random.shuffle(scenes)
    return scenes

def generate_neighborhood_mssp(solution):
    """Generates neighbors using 2-opt and 2-swap for MSSP (using scene indices)."""
    neighbors = []
    # 2-swap
    for i in range(len(solution)):
        for j in range(i + 1, len(solution)):
            neighbor_swap = solution[:]
            neighbor_swap[i], neighbor_swap[j] = neighbor_swap[j], neighbor_swap[i]
            neighbors.append(neighbor_swap)
    # 2-opt
    for i in range(len(solution)):
        for j in range(i + 1, len(solution)):
            neighbor_opt = solution[:]
            neighbor_opt[i:j+1] = reversed(neighbor_opt[i:j+1])
            neighbors.append(neighbor_opt)
    return neighbors

def identify_move_mssp(old_solution, new_solution):
    """Identifies the move (2-swap or 2-opt) made.""" # Simplified for demonstration
    if len(old_solution) != len(new_solution):
        return None
    diff_indices = []
    for i in range(len(old_solution)):
        if old_solution[i] != new_solution[i]:
            diff_indices.append(i)

    if len(diff_indices) == 2:
        return tuple(sorted(diff_indices)) # 2-swap move
    elif len(diff_indices) > 2: # Heuristic for 2-opt detection (can be improved)
        if sorted(old_solution) == sorted(new_solution): # Check if it's a reversal (permutation check)
            return "2-opt-like-move" # Not precise 2-opt identification but enough for tabu in this example
    return None

def update_particle_position_mssp(current_position, personal_best_position, global_best_position,
                             inertia_weight, cognitive_coefficient, social_coefficient):
    """Simplified PSO position update for MSSP (similar to previous PSO)."""
    new_position = current_position[:]

    indices_to_swap = list(range(len(current_position)))
    random.shuffle(indices_to_swap)

    for idx in indices_to_swap:
        if random.random() < cognitive_coefficient / (cognitive_coefficient + social_coefficient):
            if current_position[idx] != personal_best_position[idx]:
                pbest_val_at_idx = personal_best_position[idx]
                try:
                    pbest_idx_current_val = current_position.index(pbest_val_at_idx)
                    new_position[idx], new_position[pbest_idx_current_val] = new_position[pbest_idx_current_val], new_position[idx]
                except ValueError:
                    pass
        else:
            if current_position[idx] != global_best_position[idx]:
                gbest_val_at_idx = global_best_position[idx]
                try:
                    gbest_idx_current_val = current_position.index(gbest_val_at_idx)
                    new_position[idx], new_position[gbest_idx_current_val] = new_position[gbest_idx_current_val], new_position[idx]
                except ValueError:
                    pass

        if random.random() < 0.1:
            swap_idx = random.choice(indices_to_swap)
            if swap_idx != idx:
                new_position[idx], new_position[swap_idx] = new_position[swap_idx], new_position[idx]

    return new_position

def initialize_pheromones_mssp(num_scenes, initial_pheromone=1.0):
    """Initializes pheromones for MSSP."""
    return [[initial_pheromone] * num_scenes for _ in range(num_scenes)]

def construct_ant_path_mssp(start_scene, num_scenes, pheromone_levels, heuristic_function, alpha, beta):
    """Constructs ant path for MSSP."""
    path = []
    visited_scenes = set()
    current_scene = start_scene
    path.append(current_scene)
    visited_scenes.add(current_scene)

    while len(visited_scenes) < num_scenes:
        next_scene = choose_next_scene_mssp(current_scene, num_scenes, visited_scenes, pheromone_levels, heuristic_function, alpha, beta)
        if next_scene is None:
            break
        path.append(next_scene)
        visited_scenes.add(next_scene)
        current_scene = next_scene
    return path

def choose_next_scene_mssp(current_scene, num_scenes, visited_scenes, pheromone_levels, heuristic_function, alpha, beta):
    """Chooses next scene for ACO in MSSP."""
    probabilities = []
    possible_scenes = [scene for scene in range(num_scenes) if scene not in visited_scenes]

    if not possible_scenes:
        return None

    denominator = 0
    for next_scene in possible_scenes:
        pheromone = pheromone_levels[current_scene][next_scene]
        heuristic_value = heuristic_function(current_scene, next_scene)
        denominator += (pheromone**alpha) * (heuristic_value**beta)

    if denominator == 0:
        return random.choice(possible_scenes)

    for next_scene in possible_scenes:
        pheromone = pheromone_levels[current_scene][next_scene]
        heuristic_value = heuristic_function(current_scene, next_scene)
        numerator = (pheromone**alpha) * (heuristic_value**beta)
        probabilities.append(numerator / denominator)

    cumulative_probability = 0.0
    rand_val = random.random()
    for i in range(len(possible_scenes)):
        cumulative_probability += probabilities[i]
        if rand_val <= cumulative_probability:
            return possible_scenes[i]
    return possible_scenes[-1]

def update_pheromones_mssp(pheromone_levels, ant_paths, ant_path_costs, evaporation_rate, best_solution, fitness_function):
    """Updates pheromones for MSSP."""
    for i in range(len(pheromone_levels)):
        for j in range(len(pheromone_levels[i])):
            pheromone_levels[i][j] *= (1.0 - evaporation_rate)

    best_path_index = ant_path_costs.index(min(ant_path_costs))
    best_path_current_iteration = ant_paths[best_path_index]
    pheromone_deposit_amount = 1.0 / fitness_function(best_path_current_iteration)

    for i in range(len(best_path_current_iteration) - 1):
        current_scene = best_path_current_iteration[i]
        next_scene = best_path_current_iteration[i+1]
        pheromone_levels[current_scene][next_scene] += pheromone_deposit_amount

    return pheromone_levels


# --- Main Execution ---
if __name__ == "__main__":
    # Define experiment cases (min_scenes, max_scenes, min_actors, max_actors, min_locations, max_locations)
    experiment_cases = [
        (8, 15, 8, 12, 5, 8),   # Case 1: Small scale
        (20, 30, 15, 18, 10, 15),  # Case 2: Medium scale
        (30, 40, 18, 20, 15, 20),  # Case 3: Larger scale
    ]

    experiment_pipeline(experiment_cases, num_instances_per_case=3)